쿠팡 검색창 이용해서 제품명, 사용자id, 리뷰, 평점 Data 수집하기

In [1]:

import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
from tqdm import tqdm_notebook
import warnings ; warnings.filterwarnings(action='ignore')

In [2]:
# 크롬창 띄우기
driver = webdriver.Chrome("chromedriver.exe")  # 크롬 드라이버 로드
driver.get("https://www.coupang.com/")  
time.sleep(3)

In [3]:
query_text = "에어팟 프로"

In [4]:
# 검색 창에 검색어 입력 후 검색
element = driver.find_element_by_css_selector('#headerSearchKeyword')

element.send_keys(query_text)  # query_txt는 위에서 입력한 키워드
element.submit()
time.sleep(1)

In [6]:
# url 수집
urls = driver.find_elements_by_css_selector('.search-product-link')

url_list=[]
for url in urls:
    url = url.get_attribute('href')
    url_list.append(url)
    
print(len(url_list))

36


In [8]:
tmp = pd.DataFrame({'url' : url_list})
tmp.shape
#쿠팡 한페이지에 url 36개

(36, 1)

In [9]:
# 저장하기
tmp.to_excel('coupang_url_airpod.xlsx')

# 쿠팡 리뷰 블로그 내용 크롤링하기

In [11]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("coupang_url_airpod.xlsx",engine='openpyxl')
num_list = len(url_load)
#num_list = >url 주소 개수 반환
print(num_list)
url_load.head()

36


,Unnamed: 0,url
0,0,https://www.coupang.com/vp/products/332473915?...
1,1,https://www.coupang.com/vp/products/1423350307...
2,2,https://www.coupang.com/vp/products/1092323874...
3,3,https://www.coupang.com/vp/products/1077062870...
4,4,https://www.coupang.com/vp/products/4562785789...


In [12]:
# 글 띄우기
url = url_load['url'][7]
driver = webdriver.Chrome("chromedriver.exe")  # 윈도우는 "chromedriver.exe"
driver.get(url)   # 글 띄우기

In [13]:
# 제품명 크롤링
product = driver.find_element_by_css_selector('.prod-buy-header__title').text
product

'씨엘바이씨릴 에어팟프로 케이스 컬러브릭'

In [14]:
# 상품평 클릭
driver.find_element_by_css_selector('.count').click()
time.sleep(1.5)

# 상품평 수
review_num = driver.find_element_by_css_selector(".sdp-review__average__total-star__info-count")
review_num = int(review_num.text.replace(',',''))
print('상품평 수 :', review_num)

# 댓글 페이지 수
import math
page_num = review_num/5
page_num = math.ceil(page_num)  # math.ceil -> 올림
print('댓글 페이지 수 :', page_num)

상품평 수 : 149
댓글 페이지 수 : 30


In [15]:
review_overalls = driver.find_elements_by_css_selector('.sdp-review__article__list.js_reviewArticleReviewList')

In [16]:
review=review_overalls[0].text.split('\n')
review

['서*은 ',
 '2020.06.05',
 '씨엘바이씨릴 에어팟프로 케이스 컬러브릭, ASD00610, 카키',
 '튼튼하고 먼지가 안붙습니다!',
 '기존에 에어팟2세대 제품을 사용하고 있었는데,',
 '에어팟2세대 케이스 뚜껑부분이 양면테이프가 붙어있는',
 '제품이 있고, 안붙어 있는 제품도 있더라구요.',
 '',
 '에어팟2세대 케이스들은 대부분 뚜껑부분에 양면테이프가',
 '안붙어 있는 것들이 더 많았어요!',
 '',
 '그래서 에어팟프로도 뚜껑부분에 양면테이프가 없는',
 '제품을 찾으려는데, 아무래도 에어팟2세대와는 달리',
 '디자인이 많이 달라서인지 대부분의 제품들이 양면테이프가',
 '붙어있는 제품들이 더라구요ㅠㅠ',
 '',
 '제가 양면테이프 없는제품을 찾는 이유는,',
 '양면테이프로 제품을 고정하게 되면 케이스를 손쉽게 빼지 못해요...ㅠㅠ',
 '이전에 양면테이프가 뚜껑 분리를 방지한다는 말에 사용을',
 '해 본 적이 있는데 케이스를 바꿀때 뚜껑이...',
 '분해 될정도로 케이스가 안 벗겨져서 그 이후로는',
 '그런제품을 잘 안씁니다ㅠㅠ',
 '',
 '그런데 왠걸...?!',
 '이 제품은 양면테이프가 없고, 어찌보면 디자인은',
 '투박스러워 보이지만 튼튼합니다!',
 '거기다가 소재는 하드재질인듯 하고, 무광입니다!',
 '실리콘이 아니라서 먼지 붙을 일 없구요.',
 '만족스럽습니다!',
 '',
 '참고해보시라고 사진을 올려놓습니다!',
 '8명에게 도움 됨 도움이 돼요 도움 안 돼요',
 '신고하기']

In [17]:
dict = {}

# 댓글 페이지를 돌면서 리뷰 데이터 수집
for p in tqdm_notebook(range(1, page_num+1)):   

    try:
        # p번째 페이지의 리뷰 정보 수집
        for i in range(0, len(review_overalls)):        
            review_overalls = driver.find_elements_by_css_selector('.sdp-review__article__list.js_reviewArticleReviewList')
            review_num = len(review_overalls[i].text.split('\n'))
            ratings = driver.find_elements_by_css_selector(".sdp-review__article__list__info__product-info__star-orange.js_reviewArticleRatingValue")

            target_info = {}

            # user 수집
            user = review_overalls[i].text.split('\n')[0]
            target_info['user'] = user

            # date 수집
            date = review_overalls[i].text.split('\n')[1]
            target_info['date'] = date

            #  product 수집
            product = review_overalls[i].text.split('\n')[2]
            target_info['product'] = product

            # 리뷰 수집
            review = review_overalls[i].text.split('\n')[3:-2]
            target_info['review'] = review

            # 평점 수집
            rating = int(ratings[i].get_attribute('data-rating'))
            target_info['rating'] = rating     

            dict[(p-1)*5+i] = target_info
            time.sleep(1)

        print("수집 완료된 댓글 페이지 번호: ", p)
        print("페이지 마지막 유저: ", user)
            
        if p < 10:
            pages = driver.find_elements_by_css_selector(".sdp-review__article__page__num.js_reviewArticlePageBtn")
            pages[p].click()
            time.sleep(1)

        elif p%10==0:
            next_10 = driver.find_element_by_css_selector(".sdp-review__article__page__next.sdp-review__article__page__next--active.js_reviewArticlePageNextBtn")
            next_10.click()        
            time.sleep(1)

        else:
            p=p%10
            pages = driver.find_elements_by_css_selector(".sdp-review__article__page__num.js_reviewArticlePageBtn")
            pages[p].click()
            time.sleep(1)
    except:
        break
        
dict

수집 완료된 댓글 페이지 번호:  1
페이지 마지막 유저:  양*영 
수집 완료된 댓글 페이지 번호:  2
페이지 마지막 유저:  김*형 
수집 완료된 댓글 페이지 번호:  3
페이지 마지막 유저:  큐파라팡 
수집 완료된 댓글 페이지 번호:  4
페이지 마지막 유저:  묘우 
수집 완료된 댓글 페이지 번호:  5
페이지 마지막 유저:  신*영 
수집 완료된 댓글 페이지 번호:  6
페이지 마지막 유저:  한*돈 
수집 완료된 댓글 페이지 번호:  7
페이지 마지막 유저:  유*민 
수집 완료된 댓글 페이지 번호:  8
페이지 마지막 유저:  양*진 
수집 완료된 댓글 페이지 번호:  9
페이지 마지막 유저:  조*신 
수집 완료된 댓글 페이지 번호:  10
페이지 마지막 유저:  김*선 
수집 완료된 댓글 페이지 번호:  11
페이지 마지막 유저:  서*영 
수집 완료된 댓글 페이지 번호:  12
페이지 마지막 유저:  유*호 
수집 완료된 댓글 페이지 번호:  13
페이지 마지막 유저:  전*슬 
수집 완료된 댓글 페이지 번호:  14
페이지 마지막 유저:  정*환 
수집 완료된 댓글 페이지 번호:  15
페이지 마지막 유저:  김*회 
수집 완료된 댓글 페이지 번호:  16
페이지 마지막 유저:  윤*래 
수집 완료된 댓글 페이지 번호:  17
페이지 마지막 유저:  김*연 
수집 완료된 댓글 페이지 번호:  18
페이지 마지막 유저:  김*아 
수집 완료된 댓글 페이지 번호:  19
페이지 마지막 유저:  서*동 
수집 완료된 댓글 페이지 번호:  20
페이지 마지막 유저:  V*A 
수집 완료된 댓글 페이지 번호:  21
페이지 마지막 유저:  최*이 
수집 완료된 댓글 페이지 번호:  22
페이지 마지막 유저:  T*A 
수집 완료된 댓글 페이지 번호:  23
페이지 마지막 유저:  함*웅 
수집 완료된 댓글 페이지 번호:  24
페이지 마지막 유저:  배*성 
수집 완료된 댓글 페이지 번호:  25
페이지 마지막 유저:  김*인 
수집 완료된 댓글

{0: {'user': '서*은 ',
  'date': '2020.06.05',
  'product': '씨엘바이씨릴 에어팟프로 케이스 컬러브릭, ASD00610, 카키',
  'review': ['튼튼하고 먼지가 안붙습니다!',
   '기존에 에어팟2세대 제품을 사용하고 있었는데,',
   '에어팟2세대 케이스 뚜껑부분이 양면테이프가 붙어있는',
   '제품이 있고, 안붙어 있는 제품도 있더라구요.',
   '',
   '에어팟2세대 케이스들은 대부분 뚜껑부분에 양면테이프가',
   '안붙어 있는 것들이 더 많았어요!',
   '',
   '그래서 에어팟프로도 뚜껑부분에 양면테이프가 없는',
   '제품을 찾으려는데, 아무래도 에어팟2세대와는 달리',
   '디자인이 많이 달라서인지 대부분의 제품들이 양면테이프가',
   '붙어있는 제품들이 더라구요ㅠㅠ',
   '',
   '제가 양면테이프 없는제품을 찾는 이유는,',
   '양면테이프로 제품을 고정하게 되면 케이스를 손쉽게 빼지 못해요...ㅠㅠ',
   '이전에 양면테이프가 뚜껑 분리를 방지한다는 말에 사용을',
   '해 본 적이 있는데 케이스를 바꿀때 뚜껑이...',
   '분해 될정도로 케이스가 안 벗겨져서 그 이후로는',
   '그런제품을 잘 안씁니다ㅠㅠ',
   '',
   '그런데 왠걸...?!',
   '이 제품은 양면테이프가 없고, 어찌보면 디자인은',
   '투박스러워 보이지만 튼튼합니다!',
   '거기다가 소재는 하드재질인듯 하고, 무광입니다!',
   '실리콘이 아니라서 먼지 붙을 일 없구요.',
   '만족스럽습니다!',
   '',
   '참고해보시라고 사진을 올려놓습니다!'],
  'rating': 5},
 1: {'user': '이*자 ',
  'date': '2020.04.01',
  'product': '씨엘바이씨릴 에어팟프로 케이스 컬러브릭, ASD00477, 네이비',
  'review': ['에어팟 프로를 코스트코에 주3회 방문해서 겨우 겟했네요.',
   '코스트코에

In [25]:
# pages = driver.find_elements_by_css_selector(".sdp-review__article__page__num.js_reviewArticlePageBtn")
# pages[4].click()

# next_10 = driver.find_element_by_css_selector(".sdp-review__article__page__next.sdp-review__article__page__next--active.js_reviewArticlePageNextBtn")
# next_10.click()

In [19]:
# 판다스로 만들기
import pandas as pd
result_df = pd.DataFrame.from_dict(dict, 'index')

# 엑셀로 저장하기
result_df.to_excel("coupang_airpod.xlsx")